# Query engine
This technical document describes the spoc query engine, a set of classes that implements spoc's interface for querying multi-dimensional genomic data.

## Principles

### Composable pieces
Spoc's query engine consists of composable pieces that can be combined to produce an expressive query language. These pieces represent basic operations on genomic data that are easily implemented and understood on their own. This allows a great degree of flexibility, while also allowing predefined recipes that less experienced users can get started with.

### Lazy evaluation
The spoc query engine is designed with lazy evaluation as a guiding principle. This means that data queries are only executed when they are needed to minimize loading data into memory and computational overhead. To enable this, spoc queries have a construction phase, which specifies the operations to be executed and an execution phase, that actually executes the query.

## Query plans and query steps

The most important ingredient in this query language is a class that implements the `QueryStep` protocol. This protocol serves two purposes:

- It exposes a way to validate the data schema during query building
- It implements adding itself to a query

This way, query steps can be combined into a query plan that specifies the analysis to be executed. Specific examples of query steps are:

- **Snipper**: Implements selecting overlapping contacts or pixels for a set of genomic regions.
- **RegionOffsetTransformation**: Adds offset of genomic positions to regions added by snipper
- **OffsetAggregation**: Aggregates the offsets to genomic regions using an aggregation function.

### Input and output of query steps

A query step takes as input a class that implements the `GenomicData` protocol. This protocol allows retrievel of the data schema (a thin wrapper over a pandera dataframe schema) as well as the data itself. The output of a query step is again a class that ipmlements the `GenomicData` protocol to allow composition. Specific examples of possible inputs are:

- **Pixels**: Represents input pixels
- **Contacts**: Represents input contacts
- **QueryResult**: The result of a query step

### Composition of query steps

To allow specifying complex queries, query steps need to be combined. This is done using the `BasicQuery` class. It takes a query plan (a list of `QueryStep` instances) as input, exposes the `query` method, which takes input data, validates all query steps and adds them to the resulting `QueryResult` instance that is returned.

### Manifestation of results

So far, we have only talked about specifying the query to be executed, but not how to actually execute it. A `QueryResult` has a `load_result()` method that returns the manifested dataframe as a `pd.DataFrame` instance. This is the step that actually executes the specified query.

## Examples

### Selecting a subset of contacts at a single genomic position
In this example, we want to select a subset of genomic contacts at a single location. For this, we first load the required input data:

In [1]:
from spoc.query_engine import Snipper, Anchor, BasicQuery
from spoc.contacts import Contacts
import pandas as pd

contacts = Contacts.from_uri("../tests/test_files/contacts_unlabelled_2d_v2.parquet::2")

Then we specify a target region

In [2]:
target_region = pd.DataFrame({
    "chrom": ['chr1'],
    "start": [100],
    "end": [400],
})

First, we want to select all contacts where any of the fragments constituting the contact overlaps the target region. To perform this action, we use the Snipper class and pass the target region as well as an instance of the `Anchor` class. The `Anchor` dataclass allows us to specify how we want to filter contacts for region overlap. It has two attributes `mode` and `anchors`. `Anchors` indicates the positions we want to filter on (default is all positions) and `mode` specifies whether we require all positions to overlap or any position to overlap. So for example, if we want all of our two-way contacts for which any of the positions overlap, we would use `Anchor(mode='ANY', anchors=[1,2])`.

In [3]:
query_plan = [
    Snipper(target_region, anchor_mode=Anchor(mode="ANY", anchors=[1,2]))
]

A query plan is a list of qury steps that can be used in the basic query class

In [4]:
query = BasicQuery(query_plan=query_plan)

The `.query` method executes the query plan and retuns a `QueryResult` object

In [5]:
result = query.query(contacts)
result

The `.load_result` method of the `QueryResult` object can be executed using `.load_result`, which returns a `pd.DataFrame`. The resulting dataframe has additional columns that represent the regions, with which the input contacts overlapped.

In [6]:
df = result.load_result()
print(type(df))
df.filter(regex=r"chrom|start|end|id")

<class 'pandas.core.frame.DataFrame'>


,chrom_1,start_1,end_1,chrom_2,start_2,end_2,region_chrom,region_start,region_end,region_id
0,chr1,100,200,chr1,1000,2000,chr1,100,400,0
1,chr1,2000,3000,chr1,200,300,chr1,100,400,0
2,chr1,3000,4000,chr1,300,400,chr1,100,400,0


We can also restrict the positions to filter on, by passing different anchor parameters. For example, we can filter for contacts, where the first position overlaps with our target:

In [7]:
query_plan = [
    Snipper(target_region, anchor_mode=Anchor(mode="ANY", anchors=[1]))
]
BasicQuery(query_plan=query_plan)\
    .query(contacts)\
    .load_result()\
    .filter(regex=r"chrom|start|end|id")

,chrom_1,start_1,end_1,chrom_2,start_2,end_2,region_chrom,region_start,region_end,region_id
0,chr1,100,200,chr1,1000,2000,chr1,100,400,0


This time, only the first contact overlaps.

The same functionality is implemented also for the Pixels class

## Selecting a subset of contacts at multiple genomic regions
The Snipper class is also capable of selecting contacts at multiple genomic regions. Here, the behavior of `Snipper` deviates from a simple filter, because if a given contact overlaps with multiple regions, it will be returned multiple times.

Specify target regions

In [8]:
target_regions = pd.DataFrame({
    "chrom": ['chr1', 'chr1'],
    "start": [100, 150],
    "end": [400, 200],
})

In [9]:
query_plan = [
    Snipper(target_regions, anchor_mode=Anchor(mode="ANY", anchors=[1]))
]
BasicQuery(query_plan=query_plan)\
    .query(contacts)\
    .load_result()\
    .filter(regex=r"chrom|start|end|id")

,chrom_1,start_1,end_1,chrom_2,start_2,end_2,region_chrom,region_start,region_end,region_id
0,chr1,100,200,chr1,1000,2000,chr1,100,400,0
1,chr1,100,200,chr1,1000,2000,chr1,150,200,1


In this example, the contact overlapping both regions is duplicated.

The same functionality is implemented also for the pixels class.

## Calculating the offset to a target region and aggregating the result
In this example, we calculate the offset of pixels to target regions and aggregate based on the offsets. This is a very common use case in so-called pileup analyses, where we want to investigate the average behavior around regions of interest.

In [27]:
from spoc.pixels import Pixels
from spoc.query_engine import RegionOffsetTransformation
import pandas as pd
import numpy as np
from itertools import product

First we define a set of target pixels

In [21]:
def complete_synthetic_pixels():
    """Pixels that span two regions densely"""
    np.random.seed(42)
    # genomic region_1
    pixels_1 = [
        {
            "chrom": tup[0],
            "start_1": tup[1],
            "start_2": tup[2],
            "start_3": tup[3],
            "count": np.random.randint(0, 10),
        }
        for tup in product(
            ["chr1"],
            np.arange(900_000, 1_150_000, 50_000),
            np.arange(900_000, 1_150_000, 50_000),
            np.arange(900_000, 1_150_000, 50_000),
        )
    ]
    # genomic region_2
    pixels_2 = [
        {
            "chrom": tup[0],
            "start_1": tup[1],
            "start_2": tup[2],
            "start_3": tup[3],
            "count": np.random.randint(0, 10),
        }
        for tup in product(
            ["chr2"],
            np.arange(900_000, 1_150_000, 50_000),
            np.arange(900_000, 1_150_000, 50_000),
            np.arange(900_000, 1_150_000, 50_000),
        )
    ]
    return pd.concat((pd.DataFrame(pixels_1), pd.DataFrame(pixels_2)))

In [23]:
pixels = Pixels(complete_synthetic_pixels(), number_fragments=3, binsize=50_000)

Then we define the target regions we are interested in.

In [24]:
target_regions = pd.DataFrame(
        {
            "chrom": ["chr1", "chr2"],
            "start": [900_000, 900_000],
            "end": [1_100_000, 1_100_000],
        }
    )

We are then interested in selecting all contacts that are contained within these pixels and then calculate the offset to them. The selection step can be done with the `Snipper` class that we described above. The offset transformation can be done with the `OffsetTransformation` query step. This query step takes an instance of genomic data that contains regions (as defined by it's schema) and calculates the offset to all position columns. All offsets are calculated with regards to the center of each assigned region. Since genomic positions are defined by a start and end,the `OffsetTransformation` query step as an `OffsetMode` parameter that defines whether we would like to calculate the offset with regard to the start of a genomic position, the end or it's center.

In [28]:
query_plan = [
    Snipper(target_regions, anchor_mode=Anchor(mode="ANY")),
    RegionOffsetTransformation(),
]

We can then execute this query plan using the BasicQuery class. This well add an offset column to the genomic dataset returned.

In [31]:
BasicQuery(query_plan=query_plan)\
    .query(pixels)\
    .load_result()\
    .filter(regex=r"chrom|start|offset")

,chrom_1,start_1,chrom_2,start_2,chrom_3,start_3,region_chrom,region_start,offset_1,offset_2,offset_3
0,chr1,900000,chr1,900000,chr1,900000,chr1,900000,-100000.0,-100000.0,-100000.0
1,chr1,900000,chr1,900000,chr1,950000,chr1,900000,-100000.0,-100000.0,-50000.0
2,chr1,900000,chr1,900000,chr1,1000000,chr1,900000,-100000.0,-100000.0,0.0
3,chr1,900000,chr1,900000,chr1,1050000,chr1,900000,-100000.0,-100000.0,50000.0
4,chr1,900000,chr1,900000,chr1,1100000,chr1,900000,-100000.0,-100000.0,100000.0
...,...,...,...,...,...,...,...,...,...,...,...
245,chr2,1100000,chr2,1100000,chr2,900000,chr2,900000,100000.0,100000.0,-100000.0
246,chr2,1100000,chr2,1100000,chr2,950000,chr2,900000,100000.0,100000.0,-50000.0
247,chr2,1100000,chr2,1100000,chr2,1000000,chr2,900000,100000.0,100000.0,0.0
248,chr2,1100000,chr2,1100000,chr2,1050000,chr2,900000,100000.0,100000.0,50000.0


## Aggregating genomic data based on it's offset to a target region
In this example, we extend the above use-case to aggregate the results based on the offset columns added. This is a common use-case to calculate aggregate statistics for different offset levels. To achieve this, we employ the same query plan as above and extend it using the `OffsetAggregation` query step.

In [34]:
from spoc.query_engine import OffsetAggregation

The `OffsetAggregation` class requires the following parameters:
- `value_columns`: Thie specifies the value to aggregate
- `function`: The aggregation function to use. This is the enumerated type `AggregationFunction`
- `densify_output`: Whether missing offset values should be filled with empty values (specific empty value depends on the aggregation function)

Note that there are two different average functions available, `AVG` and `AVG_WITH_EMPTY`. `AVG` performs and average over all available columns, where as `AVG_WITH_EMPTY` counts missing offsets per regions as 0.

In [36]:
query_plan = [
    Snipper(target_regions, anchor_mode=Anchor(mode="ANY")),
    RegionOffsetTransformation(),
    OffsetAggregation('count'),
]

In [37]:
BasicQuery(query_plan=query_plan)\
    .query(pixels)\
    .load_result()

,offset_1,offset_2,offset_3,count
0,-100000.0,-100000.0,-100000.0,4.5
1,-100000.0,-100000.0,-50000.0,3.0
2,-100000.0,-100000.0,0.0,5.5
3,-100000.0,-100000.0,50000.0,5.0
4,-100000.0,-100000.0,100000.0,6.0
...,...,...,...,...
120,100000.0,100000.0,-100000.0,8.0
121,100000.0,100000.0,-50000.0,4.5
122,100000.0,100000.0,0.0,4.5
123,100000.0,100000.0,50000.0,4.5
